# Phonon and dielectric properties of AlAs

In this tutorial you will learn how to calculate the phonon and dielectric properties of AlAs using `aiida-vibroscopy`.

Let's get started!

In [ ]:
from local_module import load_temp_profile

# If you download this file, you can run it with your own profile.
# Put these lines instead:
# from aiida import load_profile
# load_profile()
data = load_temp_profile(
    name="polar-tutorial",
    add_computer=True,
    add_pw_code=True,
    add_phonopy_code=True,
    add_sssp=True,
)

## Importing the structure from COD

Let's import the silicon structure from the COD database via AiiDA.

In [ ]:
from aiida.plugins import DbImporterFactory

CodDbImporter = DbImporterFactory('cod')

cod = CodDbImporter()
results = cod.query(id='1540257') # AlAs 1540257
structure = results[0].get_aiida_structure() # it has 8 atoms


## The `HarmonicWorkChain`

For polar materials, the __non-analytical constants__ (NAC), i.e. Born effective charges and dielectric tensors, must be accounted when interpolating the phonon band structure. Moreover in AlAs, since it is non-centrosymmetric, a non zero $\chi^{(2)}$ is present.

Let's import the WorkChain and run it! We use the `get_builder_from_protocol` to get a __prefilled__ builder with __all inputs__. These inputs should be considered __not as converged parameters__, but as a good starting point. You may also need to tweak some parameters, e.g. add magnetization etc., depending on your case. 

In [ ]:
from aiida.orm import List
from aiida.plugins import WorkflowFactory
from aiida.engine import run_get_node

from aiida_vibroscopy.common.properties import PhononProperty

HarmonicWorkChain = WorkflowFactory("vibroscopy.phonons.harmonic")

builder = HarmonicWorkChain.get_builder_from_protocol(
    pw_code=data.pw_code,
    structure=structure,
    protocol="fast",
    phonopy_code=data.phonopy_code,
    phonon_property=PhononProperty.BANDS,
)
builder.phonon.supercell_matrix = List([1,1,1]) # this can be 2,2,2, but we save some time JUST FOR THE TUTORIAL
builder.dielectric.property = "raman" # this way it will compute also third order derivatives

results, calc = run_get_node(builder)

As we computed the __phonon dispersion__, we can easily plot it as follows:

In [ ]:
calc.outputs.output_phonopy.phonon_bands.show_mpl()

Hurray! We obtained the LO-TO splitting at the $\Gamma$ point!

## Born charges, dielectric, non-linear optical susceptibility and Raman tensors

We finally can explore all the electric field related derivatives for this material.

In [ ]:
vibro = calc.outputs.vibrational_data.numerical_accuracy_4
print("The Born effective charges: ", "\n", vibro.born_charges.round(5))

In [ ]:
print("The dielectric tensor: ", "\n", vibro.dielectric.round(5))

In [ ]:
print("The non-linear optical susceptibility tensor (pm/V): ", "\n", vibro.nlo_susceptibility.round(5))

In [ ]:
vol = vibro.get_unitcell().get_cell_volume()
print("The Raman tensors (Angstrom^2): ", "\n", vibro.raman_tensors.round(5)*vol)

## Comparison with experiments

Here the comparison with experiments. (have a look at [I. Souza et al., _Phys. Rev. Lett._, __89__, 11 (__2002__)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.89.117602) and references therein)

In [ ]:
print("Dielectric: ", "\n", "DFT: ", vibro.dielectric.round(1)[0,0], "\tExperiment: 9.6")

In [ ]:
print("Born charges: ", "\n", "DFT: ", vibro.born_charges.round(2)[0,0,0], "\tExperiment: 2.14")

In [ ]:
print("Non-linear optical susceptibility (pm/V): ", "\n", "DFT: ", vibro.nlo_susceptibility.round(0)[0,1,2], "\tExperiment: 64")

### Raman polarizability

We can also obtain the Raman polarizability tensor, both for the TO and LO modes. We can only compare with other DFT results, cause there are no available experimental data.

In [ ]:
print("TO Raman polarizability tensor (Angstrom^2): ", "\n", "DFT: ", (vibro.raman_tensors[1,0,1,2]*vol).round(2), "\tVeithen et al. (2005): 8.48")

::: {note} Why do our results differ from the one of Veithen?
:class: dropdown

The reason why we have discrepancies is mostly due to the fact we have _unconverged_ results. These tensors, being a third order derivative of the total energy, usually require a much denser k point sampling. Moreover, we also did not do the followings:
- A tight relaxation of the cell; volume can modify significantly these properties.
- Used the same functional as in other simulations, which eventually leads to different results.
- Used a proper set of inputs: it's just a tutorial, and we don't want to wait forever (i.e. 5 minutes are too much already).
:::

## Analysing the workflow

Many things happened for computing this phonon band structure. Let's inspect the process tree.

In [ ]:
%verdi process status {calc.pk}